# Digital Galton Board 
#### [V. Hunter Adams](https://vanhunteradams.com)

## Introduction

In this laboratory exercise, you will engineer a digital version of a Galton Board, which is one of the most famous physical demonstrations of order from chaos that has ever been constructed. The Galton Board delightfully combines *five* famous mathematical concepts: Bernoilli trials, the binomial distribution, the Gaussian distribution, Pascal's triangle, and the central limit theorem.

A Galton Board is composed of interleaved rows of pegs. When one drops a ball onto the top row of pegs, it bounces its way to the bottom and ultimately lands in one of N+1 bins that rest between and beneath the N lowest pegs in the board. In an idealized Galton Board, the ball bounces against a peg in each row of the board. When it strikes that peg, it has a 50% chance of bouncing to the left, and a 50% chance of bouncing to the right. If you repeat this experiment over and over and over again, the count of balls in the bins forms a nice clean binomial distribution. And as the number of rows increases and the number of trials increases, this binomial distribution, by the central limit theorem, approaches a Gaussian distribution.


<figure>
    <img align="center" width="700" height="700" src="Galton.png" alt='missing' />
    <center><figcaption>Fig. 1: Galton Board overview</figcaption></center>
</figure>

It is really startling to watch! In fact, this phenomenon so impressed Francis Galton that he stated:

*The law [the central limit theorem] would have been personified by the Greeks and deified, if they had known of it. It reigns with serenity and in complete self-effacement amidst the wildest confusion. The huger the mob, and the greater the apparent anarchy, the more perfect is its sway. It is the supreme law of Unreason. Whenever a large sample of chaotic elements are taken in hand and marshalled in the order of their magnitude, an unsuspected and most beautiful form of regularity proves to have been latent all along.*

You will build a digital version of a board, through which you will drop as many balls as you can manage at 30 frames per second. The more thoughtful and efficient your code, the more balls you will be able to animate. Every time any ball strikes a peg there must be a "thunk!" sound effect. You don't want to waste cycles doing interrupt-based audio synthesis, so you will use a DMA channel to trigger the sound effects. The balls will bounce according to collision physics. You must count and plot the number of balls that land in each bin at the bottom of the board so that we can see the central limit theorem in action. Furthermore, you will implement a playable game based on this board. The user will control the position of a paddle by means of a potentiometer, with which they will try to catch as many balls as they can. Please see the demonstration video below.

**Key concepts**: Fixed point arithmetic, optimization, direct memory access, SPI, collision physics, analog to digital converter, computer animation, VGA, PIO, alpha max beta min, overclocking, multicore parallelism, binomial distribution, Gaussian distribution, central limit theorem, Pascal's triangle.

***

## Demonstration

***

## Reading

Experience shows that students prefer these webpages short. For that reason, plese find the reading and background materials on the webpages linked below. *Please note that the information in these readings will be critical for completing the lab.*

**Math and physics background**
- The statistics of an idealized Galton Board.
- [**The physics of colliding balls, with coefficient of restitution**](./Collisions.html): Describes the physics that you will be implementing.
- [**Alpha max plus beta min algorithm**](https://en.wikipedia.org/wiki/Alpha_max_plus_beta_min_algorithm): An efficient algorithm for computing quick-and-dirty root sum of squares.

**Engineering background**
- [**VGA driver**](https://vanhunteradams.com/Pico/VGA/VGA.html): You will not be asked to implement a VGA driver, you will use the one described on this webpage. You will need to modify it if you want to overclock the RP2040.
- [**Fixed-point arithmetic**](https://vanhunteradams.com/FixedPoint/FixedPoint.html): To animate the most-possible balls, you want for your arithmetic to be as fast as possible. Floating point is too slow. Fixed-point arithmetic allows for you to use integer arithmetic (which is way faster than floating point, on this architecture) while maintaining fractional resolution.
- [**SPI interface**](https://vanhunteradams.com/Protocols/SPI/SPI.html): You will generate sound effects by means of an SPI DAC. An understanding of this interface will help you debug.
- [**DAC datasheet**](https://vanhunteradams.com/Pico/Birds/DAC.pdf): Datasheet for the MCP4822 DAC we'll be using in this lab.
- [**RP2040 datasheet**](https://datasheets.raspberrypi.com/rp2040/rp2040-datasheet.pdf): Sections on DMA, UART, Multicore, Timers, PIO, ADC, GPIO
- [**RP2040 C SDK guide**](https://www.raspberrypi.com/documentation/pico-sdk/hardware.html): Same sections as above

***

## Weekly Checkpoints
Note that these checkpoints are **cumulative**. In week 2, for example, you must have also completed all of the requirements from week 1.

**Week 1**
- Starting from [this example](https://github.com/vha3/Hunter-Adams-RP2040-Demos/tree/master/VGA_Graphics/Animation_Demo), and using the webpage on **collision physics for the digital Galton Board**, get one ball to bounce off one peg.
> - This ball should be "dropped" from the top of the screen with zero y-velocity, and a randomized (small) x-velocity.
> - The ball should accelerate downward by gravity and collide with the peg underneath, bouncing off.
> - There should be an audible sound effect, generated by DMA, when the ball strikes the peg. Modify the DMA channels from [this example](https://github.com/vha3/Hunter-Adams-RP2040-Demos/tree/master/Audio/e_DMA_Demo) to generate the sound effect.
> - When the ball exits the bottom of the screen, it should automatically drop again from the top.
> - Use all the default parameter values recommended on the Galton Board collision physics page.

**Week 2**
- Generate the 16-row Galton Board, and animate at least 10 balls falling through it. Each ball should generate a sound effect when it collides with a new peg, and each should be re-spawned when it falls thru the bottom of the screen.
- On the VGA screen, display the current number of balls being animated, and the total number that have fallen through the board since reset.
- Animate a horizontal line, as wide as the space between the Galton Board pegs, underneath the bottom row of the Galton Board. The user should be able to move the line with a potentiometer.
- Generate a histogram underneath the Galton Board that shows the number of balls that have fallen through each pair of pegs in the bottom row.

**Week 3**
- At reset, the maximum number of balls which you can animate at 30fps should be dropped into the top of the Galton Board.
- Each of these balls should generate a "thunk" sound when it hits a new peg.
- By turning a potentiometer, the user should be able to increase/decrease the number of balls being animated.
- Upon a button push, the potentiometer should *instead* control the position of a paddle beneath the Galton Board, and a new game should start. Users will try to catch as many balls as they can with this paddle for a 30-second game. After the game time expires, the potentiometer will again control the number of balls being animated. Pushing the button during the game cancels the game.
- A histogram, normalized to the available vertical space beneath the Galton Board, should show the number of balls that have fallen through each pair of pegs in the bottom row.
- The VGA screen should display:
> - The current number of balls being animated.
> - The total number that have fallen through the board since reset.
> - During gameplay: the current score (balls caught with paddle) by the user
> - The highscore, since the system was powered on.
> - The time remaining in the game during gameplay.
- If the 30fps deadline is missed, the LED should turn on.

***

## Lab Report
Your lab report should include all the sections mentioned on the [policy page](https://vanhunteradams.com/Pico/CourseMaterials/Policy.html), and also answer the following questions:
- Our physical model of the Galton Board differs from the idealized model. In order for the Central Limit Theorem to hold, all random numbers must be **independent**, **identically distributed**, and have **finite variance.** Which of these assumptions hold for our physical Galton Board model, and which do not? Why do we still see a distribution that approaches a Gaussian? (You needn't prove anything here, just try to identify the generalization of the CLT that is responsible for this).
- The Gaussian approximates the distribution under the Galton board *better* with *more rows in the board*. Why?
- For your 16-row Galton Board, what is the mean and standard deviation of the Gaussian which approximates the resulting distribution (under the assumption that your Galton Board is "ideal")?

***